In [30]:
from rdflib import Graph,URIRef,Namespace,BNode,Literal,XSD
from rdflib.namespace import XSD,RDF
import pandas as pd
import argparse
from collections import defaultdict
from datetime import datetime
import json
from rdflib import URIRef
from collections import defaultdict
import json



In [2]:
graph = Graph()
directory = 'sampleTSS.ttl'
print("Started loading graph...")
graph.parse(directory, format="turtle",publicID="https://example.org/")
print("Graph loaded successfully.")

Started loading graph...
Graph loaded successfully.


In [ ]:

#Get unique time series snippets
snippet_set = set()

get_snippet_query = '''
        PREFIX sosa: <http://www.w3.org/ns/sosa/>
        PREFIX tss:  <https://w3id.org/tss#>
        PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

        SELECT DISTINCT ?snippet ?PtsObj
        WHERE{
            ?snippet a tss:Snippet .
            ?snippet tss:points ?PtsObj .
        }
        '''
print('Started identifying unique sensors')
for snippet, PtsObj in graph.query(get_snippet_query):
    snippet_set.add(snippet)   # keep adding the snippet URI
    print(f"Snippet: {snippet}, PtsObj: {PtsObj}")
    print(type(PtsObj))

print('Snippets identified successfully')


SyntaxError: invalid syntax (2296010491.py, line 24)

In [33]:
# Nested dict: subject -> predicate -> list of objects
#results = defaultdict(lambda: defaultdict(list))

#Create RDF
prefix_tss = Namespace('https://w3id.org/tss#')
prefix_ex  = Namespace('http://example.org/')
prefix_sosa  = Namespace('http://www.w3.org/ns/sosa/')
prefix_xsd  = Namespace('http://www.w3.org/2001/XMLSchema#')
final_graph = Graph()
final_graph.bind('tss', prefix_tss)
final_graph.bind('ex', prefix_ex)
final_graph.bind('sosa', prefix_sosa)
final_graph.bind('xsd', prefix_xsd)
print('Started creating final graph')


get_snippet_query = '''
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX tss:  <https://w3id.org/tss#>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

SELECT ?snippet ?P ?O ?about ?aboutP ?aboutO
WHERE {
  ?snippet a tss:Snippet .
  ?snippet ?P ?O .

  OPTIONAL {
    ?snippet tss:about ?about .
    ?about ?aboutP ?aboutO .
  }
}

        '''
tss_points = URIRef("https://w3id.org/tss#points")
tss_Snippet = URIRef("https://w3id.org/tss#Snippet")
tss_PointTemplate = URIRef("https://w3id.org/tss#PointTemplate")


for subj, pred, obj, about, aboutP, aboutO in graph.query(get_snippet_query):
    # store snippet triples
    if subj and pred and obj:
        if pred == tss_points: #this is where i should focus on derefrencing the json object
          #results[subj][pred].append(obj)
          parsed = json.loads(str(obj)) #json array
          for point in parsed:
            json_time = point['time']
            json_value = point['value']
            #now convert them from strings and add them to final graph
            json_time = Literal(json_time,datatype=XSD.dateTime)
            json_value = Literal(json_value,datatype=XSD.decimal) 

            final_graph.add((subj,URIRef("http://www.w3.org/ns/sosa/resultTime"),json_time))
            final_graph.add((subj,URIRef("http://www.w3.org/ns/sosa/hasSimpleResult"),json_value))

          #final_graph.add((subj, pred, obj)) #instead of adding the entire json object, will break it down and add each element as object.
        
        if obj == tss_Snippet:
            continue

    # store about triples
    if about and aboutP and aboutO:
        if aboutO != tss_PointTemplate:
          #results[subj][aboutP].append(aboutO)
          final_graph.add((subj, aboutP, aboutO))


Started creating final graph


In [36]:
print('Started writing file to disk')
final_graph.serialize('generated_TSS2RDF.ttl', format="turtle")
print('File written successfully')

Started writing file to disk
File written successfully
